# 图像分类

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak

In [2]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[2] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

In [3]:
gpus[:2]

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

## 一个简单例子

第一步是准备数据。 这里我们以MNIST数据集为例

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

(60000, 28, 28)
(60000,)
[5 0 4]


第二步是运行ImageClassifier。 建议对更复杂的数据集进行更多试验。 这只是 MNIST 的快速演示，因此我们将 max_trials 设置为 1。出于同样的原因，我们将 epochs 设置为 10。您也可以为自适应数量的 epochs 不指定 epochs。

In [10]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 04m 25s]
val_loss: 0.03918910399079323

Best val_loss So Far: 0.03918910399079323
Total elapsed time: 00h 04m 25s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.1584 - accuracy: 0.9508
Epoch 2/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0745 - accuracy: 0.9771
Epoch 3/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0582 - accuracy: 0.9821
Epoch 4/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0503 - accuracy: 0.9842
Epoch 5/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0446 - accuracy: 0.9861
Epoch 6/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0420 - accuracy: 0.9863
Epoch 7/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0365 - accuracy: 0.9886
Epoch 8/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.0370 - accuracy: 0

In [14]:
# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)

313/313 [==============================] - 1s 3ms/step
[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]


In [15]:
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

313/313 [==============================] - 4s 14ms/step - loss: 0.0362 - accuracy: 0.9891
[0.036244697868824005, 0.9890999794006348]


## 验证数据

默认情况下，AutoKeras 使用最后 20% 的训练数据作为验证数据。 如下例所示，您可以使用validation_split 来指定百分比。

In [16]:
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)

您还可以使用自己的验证集，而不是使用validation_data 将其从训练数据中分离出来。

In [17]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=10,
)

## 自定义搜索空间

对于高级用户，您可以使用 AutoModel 而不是 ImageClassifier 来自定义您的搜索空间。 您可以为一些高级配置配置 ImageBlock，例如，block_type 为要搜索的神经网络类型，normalize 为是否进行数据归一化，augment 为是否进行数据增强。 您也可以不指定这些参数，这将使不同的选择自动调整。 有关详细信息，请参阅以下示例。

In [5]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 31m 31s]
val_loss: 0.09327429533004761

Best val_loss So Far: 0.09327429533004761
Total elapsed time: 00h 31m 31s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 210s 104ms/step - loss: 0.6322 - accuracy: 0.8437
Epoch 2/10
1875/1875 [==============================] - 195s 104ms/step - loss: 0.4301 - accuracy: 0.9044
Epoch 3/10
1875/1875 [==============================] - 194s 104ms/step - loss: 0.2932 - accuracy: 0.9477
Epoch 4/10
1875/1875 [==============================] - 193s 103ms/step - loss: 0.3473 - accuracy: 0.9196
Epoch 5/10
1875/1875 [==============================] - 191s 102ms/step - loss: 0.3149 - accuracy: 0.9367
Epoch 6/10
1875/1875 [==============================] - 193s 103ms/step - loss: 0.2574 - accuracy: 0.9401
Epoch 7/10
1875/1875 [==============================] - 193s 103ms/step - loss: 0.1488 - accuracy: 0.9639
Epoch 8/10
1875/1875 [==============================] - 190s 101ms/step - loss: 0.11

/public/huangwei/miniconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


AutoModel 的用法类似于 Keras 的函数式 API。 基本上，您正在构建一个图，其边是块，节点是块的中间输出。 使用 output_node = ak.some_block(input_node) 添加从 input_node 到 output_node 的边。

您甚至还可以使用更细粒度的块来进一步自定义搜索空间。 请参阅以下示例。

In [6]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 13m 18s]
val_loss: 0.07811136543750763

Best val_loss So Far: 0.07811136543750763
Total elapsed time: 00h 13m 18s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 86s 43ms/step - loss: 0.3004 - accuracy: 0.9138
Epoch 2/10
1875/1875 [==============================] - 80s 43ms/step - loss: 0.1941 - accuracy: 0.9481
Epoch 3/10
1875/1875 [==============================] - 80s 43ms/step - loss: 0.1303 - accuracy: 0.9639
Epoch 4/10
1875/1875 [==============================] - 79s 42ms/step - loss: 0.1240 - accuracy: 0.9661
Epoch 5/10
1875/1875 [==============================] - 80s 43ms/step - loss: 0.0994 - accuracy: 0.9716
Epoch 6/10
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0752 - accuracy: 0.9781
Epoch 7/10
1875/1875 [==============================] - 79s 42ms/step - loss: 0.0727 - accuracy: 0.9794
Epoch 8/10
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0591 - accuracy: 0

/public/huangwei/miniconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


## 数据格式

AutoKeras ImageClassifier 对于数据格式非常灵活。

对于图像，它接受带和不带通道维度的数据格式。 MNIST 数据集中的图像没有通道维度。 每个图像都是一个形状为 (28, 28) 的矩阵。 AutoKeras 也接受最终带有通道维度的三个维度的图像，例如 (32, 32, 3), (28, 28, 1)。

对于分类标签，AutoKeras 接受普通标签（即字符串或整数）和单热编码标签（即 0 和 1 的向量）。

因此，如果您按以下方式准备数据，ImageClassifier 应该仍然有效。

In [18]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have the channel dimension.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

In [21]:
# One-hot encode the labels.
eye = np.eye(10)
y_train = eye[y_train]
y_test = eye[y_test]

In [22]:
print(x_train.shape)  # (60000, 28, 28, 1)
print(y_train.shape)  # (60000, 10)
print(y_train[:3])
# array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
#        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

(60000, 28, 28, 1)
(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


我们还支持对训练数据使用 tf.data.Dataset 格式。

In [23]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,)))
test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,)))

clf = ak.ImageClassifier(overwrite=True, max_trials=1)

In [24]:
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set, epochs=10)

Trial 1 Complete [00h 04m 17s]
val_loss: 0.03819820284843445

Best val_loss So Far: 0.03819820284843445
Total elapsed time: 00h 04m 17s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1591 - accuracy: 0.9517
Epoch 2/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0732 - accuracy: 0.9776
Epoch 3/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0584 - accuracy: 0.9820
Epoch 4/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0525 - accuracy: 0.9832
Epoch 5/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0437 - accuracy: 0.9861
Epoch 6/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0403 - accuracy: 0.9871
Epoch 7/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0375 - accuracy: 0.9883
Epoch 8/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0343 - accuracy: 0

In [25]:
# Predict with the best model.
predicted_y = clf.predict(test_set)

313/313 [==============================] - 1s 3ms/step


In [26]:
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))

313/313 [==============================] - 4s 12ms/step - loss: 0.0312 - accuracy: 0.9903
[0.031231015920639038, 0.9902999997138977]
